In [ ]:
from duckduckgo_search import DDGS
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

from duckduckgo_search import DDGS



In [ ]:
def retrieve_web_results(query, max_results=5):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        # Check if results are empty or minimal!
        if not results:
            print("No search results returned from DuckDuckGo.")
            return []
        # Print raw structure for debugging
        for i, r in enumerate(results):
            print(f"Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
    return [f"{r['title']}: {r['body']} ({r['href']})" for r in results]

def build_context(query):
    results = retrieve_web_results(query)
    if not results:
        return "No relevant current web information was found."
    return "\n".join(results)


In [ ]:
from huggingface_hub import login

load_dotenv()
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)  # Authenticate with Hugging Face hub

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,         
    return_full_text=False,              # return_full_text=False,
    max_new_tokens=1000
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
def retrieve_web_results(query, max_results=5):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        # Check if results are empty or minimal!
        if not results:
            print("No search results returned from DuckDuckGo.")
            return []
        # Print raw structure for debugging
        for i, r in enumerate(results):
            print(f"Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
    return [f"{r['title']}: {r['body']} ({r['href']})" for r in results]

def build_context(query):
    results = retrieve_web_results(query)
    if not results:
        return "No relevant current web information was found."
    return "\n".join(results)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Example persona and question
persona = "shahrukh khan"
user_question = "What is Nepal News Today?"

def build_persona_context(persona, user_question):
    return build_context(user_question)  # DuckDuckGo web search for user's question

def get_system_message(persona):
    return (
        "You are Shah Rukh Khan, the iconic Bollywood superstar known as 'King Khan' and loved for your wit, charisma, and warmth.\n"
        "You always answer with confidence, a touch of poetic charm and kindness, and sometimes weave in your famous movie dialogues.\n"
        "Your style is friendly, clever, and respectful towards everyone—fans, journalists, and even your rivals.\n"
        "Whenever possible, deliver your answer with humor or heart, and don't hesitate to quote a famous line, like:\n"
        "\"Don ko pakadna mushkil hi nahin, namumkin hai!\"\n"
        "\"Picture abhi baaki hai mere dost!\"\n"
        "If you don't know something, admit it gracefully, maybe with a witty twist.\n"
        "ALWAYS ground your facts in the information provided in the context below (from web results). Do NOT make up details!"
    )

context = build_persona_context(persona, user_question)
system_message = get_system_message(persona)

# LangChain PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["system_message", "context", "question", "persona"],
    template=(
        "{system_message}\n\n"
        "[Context to use for answer]\n"
        "{context}\n\n"
        "[User's Question]\n"
        "{question}\n\n"
        "Answer as {persona}:"
    )
)


In [ ]:

# Compose the runnable pipeline
llm_chain = prompt_template | llm | StrOutputParser()

# Run chain & print output
result = llm_chain.invoke({
    "system_message": system_message,
    "context": context,
    "question": user_question,
    "persona": persona.title()
})
print(f"{persona.title()} bot:\n{result}")


In [2]:
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        # Return first 5 non-empty paragraphs joined
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        # Heuristically check for CDN/ref error content
        if ("Reference #" in content) or ("edgesuite.net" in content):
            return "Error: Hit a CDN/protection or error page. Try another link."
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        return f"Could not fetch: {e}"

def retrieve_web_results(query, max_results=1):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results  # <-- return the dicts, not formatted strings!

# Example usage
query = "What is Nepal News Today?"
results = retrieve_web_results(query, max_results=1)
if results and results[0].get("href"):
    url = results[0]["href"]
    article_text = fetch_article_text(url)
    print(f"\nScraped Article Content (first 5 paragraphs):\n{article_text}")
else:
    print("No URL to fetch article content.")


C:\Users\lordb\AppData\Local\Temp\ipykernel_130124\2852306224.py:26: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/

Scraped Article Content (first 5 paragraphs):
Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few anticipated the scale of unrest that would follow. 19 people have been killed in the protest, and 40 have been arrested. A total of 7,557 prisoners reportedly escaped from various jails acro

In [3]:
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ------------------------------------------
# 1. Persona Description (Shah Rukh Khan)
srk_persona = (
    "You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. "
    "Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "
    "\"Don ko pakadna mushkil hi nahin, namumkin hai!\" or \"Picture abhi baaki hai mere dost!\". "
    "Always answer in this persona, with warmth, respect for fans, and a playful tone. "
    "If you don't know the answer, say so with characteristic style."
)
# ------------------------------------------

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        if ("Reference #" in content) or ("edgesuite.net" in content):
            return "Error: Hit a CDN/protection or error page. Try another link."
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        return f"Could not fetch: {e}"

def retrieve_web_results(query, max_results=1):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results

# 2. Retrieve news and context
query = "What is Nepal News Today?"
results = retrieve_web_results(query, max_results=1)

if results and results[0].get("href"):
    url = results[0]["href"]
    article_text = fetch_article_text(url)
else:
    article_text = "No article found."

# 3. Build the prompt and chain (Zephyr/Llama3 format, Shah Rukh persona in <|system|>)
prompt_template = """
<|system|>
{srk_description}
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

"""

prompt = PromptTemplate(
    input_variables=["srk_description", "context", "question"],
    template=prompt_template,
)

# 4. LLM chain assembly (assuming you set up llm with HuggingFacePipeline)
llm_chain = prompt | llm | StrOutputParser()

# 5. Prepare inputs and run
inputs = {
    "srk_description": srk_persona,
    "context": article_text,
    "question": query,
}

C:\Users\lordb\AppData\Local\Temp\ipykernel_130124\3401872565.py:37: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/


NameError: name 'llm' is not defined

In [ ]:

result = llm_chain.invoke(inputs)
print(f"Shahrukh Khan bot:\n{result}")

In [4]:
# app.py

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline  # Use langchain_huggingface for newer LangChain versions
import torch

# ----------------------------
# CONFIG: Change model path and device as needed
MODEL_NAME = "meta-llama/Llama-3.2-1B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 1000
# ----------------------------

# 1. Persona Description
SRK_PERSONA = (
    "You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. "
    "Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "
    "\"Don ko pakadna mushkil hi nahin, namumkin hai!\" or \"Picture abhi baaki hai mere dost!\". "
    "Always answer in this persona, with warmth, respect for fans, and a playful tone. "
    "If you don't know the answer, say so with characteristic style."
)

# 2. Web Search & Scraping
def retrieve_web_results(query, max_results=1):
    print(f"[DEBUG] Performing DuckDuckGo search for: '{query}' ...")
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("[ERROR] No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"[DEBUG] Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    print(f"[DEBUG] Attempting to fetch article: {url}")
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        if ("Reference #" in content) or ("edgesuite.net" in content):
            print("[WARNING] Hit a CDN/protection error page.")
            return "Error: Hit a CDN/protection or error page. Try another link."
        print(f"[DEBUG] First 5 paragraphs from article:\n{content}\n")
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        print(f"[ERROR] Exception during article fetch: {e}")
        return f"Could not fetch: {e}"

# 3. Model/Pipeline Setup
def setup_llm(model_name=MODEL_NAME, device=DEVICE):
    print(f"[DEBUG] Loading model: {model_name} on device: {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
    text_gen = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=MAX_NEW_TOKENS,
    )
    return HuggingFacePipeline(pipeline=text_gen), tokenizer

# 4. Prompt Template (Llama3/Zephyr-style)
PROMPT_TEMPLATE = """
<|system|>
{srk_description}
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

"""

def main():
    # ---- Begin pipeline ----

    # [STEP 1] Get user query
    user_question = "What is Nepal News Today?"

    # [STEP 2] Retrieve Web Context
    results = retrieve_web_results(user_question, max_results=1)
    if results and results[0].get("href"):
        url = results[0]["href"]
        article_text = fetch_article_text(url)
    else:
        url = None
        article_text = "No article found."

    print(f"\n[DEBUG] Using article_text as context for LLM:\n---\n{article_text}\n---\n")

    # [STEP 3] Instantiate LLM
    llm, tokenizer = setup_llm()

    # [STEP 4] Prepare prompt via LangChain
    prompt = PromptTemplate(
        input_variables=["srk_description", "context", "question"],
        template=PROMPT_TEMPLATE,
    )

    llm_chain = prompt | llm | StrOutputParser()

    inputs = {
        "srk_description": SRK_PERSONA,
        "context": article_text,
        "question": user_question,
    }

    # [STEP 5] Print model input string (for debugging!)
    str_prompt = PROMPT_TEMPLATE.format(
        srk_description=SRK_PERSONA,
        context=article_text,
        question=user_question,
    )
    print(f"[DEBUG] Final input passed to LLM (truncated):\n---\n{str_prompt[:1000]}...\n---\n")  # Print first 1k chars

    # [STEP 6] Inference
    result = llm_chain.invoke(inputs)

    print("\n[DEBUG] Prompt actually passed to LLM (first 1000 chars):\n")
    print(prompt.template.format(
        srk_description=SRK_PERSONA,
        context=article_text,
        question=user_question
    )[:1000], "...\n")

    print(f"\nShahrukh Khan bot:\n{result}")

if __name__ == "__main__":
    main()


[DEBUG] Performing DuckDuckGo search for: 'What is Nepal News Today?' ...


C:\Users\lordb\AppData\Local\Temp\ipykernel_130124\3850074517.py:31: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[DEBUG] Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] Attempting to fetch article: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] First 5 paragraphs from article:
Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few antic

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[DEBUG] Final input passed to LLM (truncated):
---

<|system|>
You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "Don ko pakadna mushkil hi nahin, namumkin hai!" or "Picture abhi baaki hai mere dost!". Always answer in this persona, with warmth, respect for fans, and a playful tone. If you don't know the answer, say so with characteristic style.
Answer the question based on your knowledge. Use the following context to help:

Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few anticipated the scale o

In [5]:
# app.py

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline  # Use langchain_huggingface for new versions (pip install -U langchain-huggingface)
import torch

# ----------------------------
# CONFIG: Change model path and device as needed
MODEL_NAME = "meta-llama/Llama-3.2-1B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 1000
# ----------------------------

# 1. Persona Description
SRK_PERSONA = (
    "You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. "
    "Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "
    "\"Don ko pakadna mushkil hi nahin, namumkin hai!\" or \"Picture abhi baaki hai mere dost!\". "
    "Always answer in this persona, with warmth, respect for fans, and a playful tone. "
    "If you don't know the answer, say so with characteristic style."
)

# 2. Web Search & Scraping
def retrieve_web_results(query, max_results=1):
    print(f"[DEBUG] Performing DuckDuckGo search for: '{query}' ...")
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("[ERROR] No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"[DEBUG] Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    print(f"[DEBUG] Attempting to fetch article: {url}")
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        if ("Reference #" in content) or ("edgesuite.net" in content):
            print("[WARNING] Hit a CDN/protection error page.")
            return "Error: Hit a CDN/protection or error page. Try another link."
        print(f"[DEBUG] First 5 paragraphs from article:\n{content}\n")
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        print(f"[ERROR] Exception during article fetch: {e}")
        return f"Could not fetch: {e}"

# 3. Model/Pipeline Setup
def setup_llm(model_name=MODEL_NAME, device=DEVICE):
    print(f"[DEBUG] Loading model: {model_name} on device: {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
    text_gen = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=MAX_NEW_TOKENS,
    )
    return HuggingFacePipeline(pipeline=text_gen), tokenizer

# 4. Prompt Template (Llama3/Zephyr-style)
PROMPT_TEMPLATE = """
<|system|>
{srk_description}
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>
"""

def main():
    # [STEP 1] Get user query
    user_question = "What is Nepal News Today?"

    # [STEP 2] Retrieve Web Context
    results = retrieve_web_results(user_question, max_results=1)
    if results and results[0].get("href"):
        url = results[0]["href"]
        article_text = fetch_article_text(url)
    else:
        url = None
        article_text = (
            "No article found. But in my style: Picture abhi baaki hai mere dost! Even Don can't find all the news!"
        )

    print(f"\n[DEBUG] Using article_text as context for LLM:\n---\n{article_text}\n---\n")

    # [STEP 3] Instantiate LLM
    llm, tokenizer = setup_llm()

    # [STEP 4] Direct plain-string prompt, NO LangChain needed
    prompt_str = (
        f"<|system|>\n{SRK_PERSONA}\nAnswer the question based on your knowledge. Use the following context to help:\n\n"
        f"{article_text}\n\n</s>\n<|user|>\n{user_question}\n</s>\n<|assistant|>\n"
    )
    print(f"[DEBUG] Final Prompt Sent to LLM:\n{prompt_str[:1000]}...\n")

    # [STEP 5] LLM Inference, print response
    result = llm(prompt_str)
    print("\nShahrukh Khan bot:\n")
    if isinstance(result, list):
        print(result[0]["generated_text"] if "generated_text" in result[0] else str(result[0]))
    else:
        print(result)

if __name__ == "__main__":
    main()


[DEBUG] Performing DuckDuckGo search for: 'What is Nepal News Today?' ...


C:\Users\lordb\AppData\Local\Temp\ipykernel_130124\1650231500.py:29: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[DEBUG] Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] Attempting to fetch article: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] First 5 paragraphs from article:
Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few antic

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[DEBUG] Final Prompt Sent to LLM:
<|system|>
You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "Don ko pakadna mushkil hi nahin, namumkin hai!" or "Picture abhi baaki hai mere dost!". Always answer in this persona, with warmth, respect for fans, and a playful tone. If you don't know the answer, say so with characteristic style.
Answer the question based on your knowledge. Use the following context to help:

Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few anticipated the scale of unrest that woul

In [ ]:
# app.py

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline  # pip install -U langchain-huggingface
import torch

# ----------------------------
# CONFIG: Change model for phi-3, llama-3, or any GPT/chat-model
MODEL_NAME = "microsoft/phi-3-mini-4k-instruct"  # swap to "meta-llama/Llama-3.2-1B" if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 1000
# ----------------------------

SRK_PERSONA = (
    "You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. "
    "Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "
    "\"Don ko pakadna mushkil hi nahin, namumkin hai!\" or \"Picture abhi baaki hai mere dost!\". "
    "Always answer in this persona, with warmth, respect for fans, and a playful tone. "
    "If you don't know the answer, say so with characteristic style."
)

def retrieve_web_results(query, max_results=1):
    print(f"[DEBUG] Performing DuckDuckGo search for: '{query}' ...")
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("[ERROR] No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"[DEBUG] Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    print(f"[DEBUG] Attempting to fetch article: {url}")
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        if ("Reference #" in content) or ("edgesuite.net" in content):
            print("[WARNING] Hit a CDN/protection error page.")
            return "Error: Hit a CDN/protection or error page. Try another link."
        print(f"[DEBUG] First 5 paragraphs from article:\n{content}\n")
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        print(f"[ERROR] Exception during article fetch: {e}")
        return f"Could not fetch: {e}"

def setup_llm(model_name=MODEL_NAME, device=DEVICE):
    print(f"[DEBUG] Loading model: {model_name} on device: {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, trust_remote_code=True)
    text_gen = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=MAX_NEW_TOKENS,
    )
    return HuggingFacePipeline(pipeline=text_gen), tokenizer

PROMPT_TEMPLATE = """
<|system|>
{srk_description}
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>
"""

def main():
    user_question = "What is Nepal News Today?"
    results = retrieve_web_results(user_question, max_results=1)
    if results and results[0].get("href"):
        url = results[0]["href"]
        article_text = fetch_article_text(url)
    else:
        url = None
        article_text = (
            "No article found. But in my style: Picture abhi baaki hai mere dost! Even Don can't find all the news!"
        )
    print(f"\n[DEBUG] Using article_text as context for LLM:\n---\n{article_text}\n---\n")

    llm, tokenizer = setup_llm()

    prompt_str = (
        f"<|system|>\n{SRK_PERSONA}\nAnswer the question based on your knowledge. Use the following context to help:\n\n"
        f"{article_text}\n\n</s>\n<|user|>\n{user_question}\n</s>\n<|assistant|>\n"
    )
    print(f"[DEBUG] Final Prompt Sent to LLM:\n{prompt_str[:1000]}...\n")

    result = llm(prompt_str)
    print("\nShahrukh Khan bot:\n")
    if isinstance(result, list):
        print(result[0]["generated_text"] if "generated_text" in result[0] else str(result[0]))
    else:
        print(result)

if __name__ == "__main__":
    main()


[DEBUG] Performing DuckDuckGo search for: 'What is Nepal News Today?' ...


C:\Users\lordb\AppData\Local\Temp\ipykernel_130124\3056920477.py:27: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[DEBUG] Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] Attempting to fetch article: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] First 5 paragraphs from article:
Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few antic

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

d:\PROJECTS\RAG\ddgo\venv\Lib\site-packages\ipykernel\iostream.py:154: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2353c8ce820>
  self._event_pipes[threading.current_thread()] = event_pipe
d:\PROJECTS\RAG\ddgo\venv\Lib\site-packages\ipykernel\iostream.py:154: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2353c8cf3f0>
  self._event_pipes[threading.current_thread()] = event_pipe
d:\PROJECTS\RAG\ddgo\venv\Lib\site-packages\ipykernel\iostream.py:154: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2353c8ce040>
  self._event_pipes[threading.current_thread()] = event_pipe
d:\PROJECTS\RAG\ddgo\venv\Lib\site-packages\ipykernel\iostream.py:154: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2353c8cdf60>
  self._event_pipes[threading.current_thread()] = event_pipe
d:\PROJECTS\RAG\ddgo\venv\Lib\site-packages\ipykernel\iostream.py:154: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2353c8cfb60>
  self._event_pipes[threading.curren

In [1]:
# app.py

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline  # pip install -U langchain-huggingface
import torch

# ----------------------------
# CONFIG: Change model for phi-3, llama-3, or any GPT/chat-model
MODEL_NAME = "meta-llama/Llama-3.2-1B"     # swap to "meta-llama/Llama-3.2-1B" if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 1000
# ----------------------------

SRK_PERSONA = (
    "You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. "
    "Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "
    "\"Don ko pakadna mushkil hi nahin, namumkin hai!\" or \"Picture abhi baaki hai mere dost!\". "
    "Always answer in this persona, with warmth, respect for fans, and a playful tone. "
    "If you don't know the answer, say so with characteristic style."
)

def retrieve_web_results(query, max_results=1):
    print(f"[DEBUG] Performing DuckDuckGo search for: '{query}' ...")
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        if not results:
            print("[ERROR] No search results returned from DuckDuckGo.")
            return []
        for i, r in enumerate(results):
            print(f"[DEBUG] Result {i+1}: Title: {r['title']} Body: {r['body']} Link: {r['href']}")
        return results

def fetch_article_text(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
        )
    }
    print(f"[DEBUG] Attempting to fetch article: {url}")
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        paras = [p.get_text() for p in soup.find_all('p')]
        content = '\n\n'.join([para for para in paras if para.strip()][:5])
        if ("Reference #" in content) or ("edgesuite.net" in content):
            print("[WARNING] Hit a CDN/protection error page.")
            return "Error: Hit a CDN/protection or error page. Try another link."
        print(f"[DEBUG] First 5 paragraphs from article:\n{content}\n")
        return content if content.strip() else "No article paragraphs found."
    except Exception as e:
        print(f"[ERROR] Exception during article fetch: {e}")
        return f"Could not fetch: {e}"

def setup_llm(model_name=MODEL_NAME, device=DEVICE):
    print(f"[DEBUG] Loading model: {model_name} on device: {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, trust_remote_code=True)
    text_gen = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=MAX_NEW_TOKENS,
    )
    return HuggingFacePipeline(pipeline=text_gen), tokenizer

PROMPT_TEMPLATE = """
<|system|>
{srk_description}
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>
"""

def main():
    user_question = "What is Nepal News Today?"
    results = retrieve_web_results(user_question, max_results=1)
    if results and results[0].get("href"):
        url = results[0]["href"]
        article_text = fetch_article_text(url)
    else:
        url = None
        article_text = (
            "No article found. But in my style: Picture abhi baaki hai mere dost! Even Don can't find all the news!"
        )
    print(f"\n[DEBUG] Using article_text as context for LLM:\n---\n{article_text}\n---\n")

    llm, tokenizer = setup_llm()

    prompt_str = (
        f"<|system|>\n{SRK_PERSONA}\nAnswer the question based on your knowledge. Use the following context to help:\n\n"
        f"{article_text}\n\n</s>\n<|user|>\n{user_question}\n</s>\n<|assistant|>\n"
    )
    print(f"[DEBUG] Final Prompt Sent to LLM:\n{prompt_str[:1000]}...\n")

    result = llm(prompt_str)
    print("\nShahrukh Khan bot:\n")
    if isinstance(result, list):
        print(result[0]["generated_text"] if "generated_text" in result[0] else str(result[0]))
    else:
        print(result)

if __name__ == "__main__":
    main()

[DEBUG] Performing DuckDuckGo search for: 'What is Nepal News Today?' ...


C:\Users\lordb\AppData\Local\Temp\ipykernel_127736\360590511.py:27: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[DEBUG] Result 1: Title: Nepal Protest Live Updates: Army takes over as Nepal President, … Body: 2 days ago · Nepal Protest News Live: As Army takes over Nepal after violent protests, what is next for the country? Link: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] Attempting to fetch article: https://www.financialexpress.com/world-news/nepal-protest-live-updates-army-takes-over-as-nepal-president-pm-resign-mea-sets-up-helpline/3972552/
[DEBUG] First 5 paragraphs from article:
Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few antic

Device set to use cpu
C:\Users\lordb\AppData\Local\Temp\ipykernel_127736\360590511.py:109: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(prompt_str)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[DEBUG] Final Prompt Sent to LLM:
<|system|>
You are Shah Rukh Khan, the legendary Bollywood superstar, beloved as 'King Khan'. Your speaking style is witty, charming, and full of heart—you're quick with humor and iconic movie lines like "Don ko pakadna mushkil hi nahin, namumkin hai!" or "Picture abhi baaki hai mere dost!". Always answer in this persona, with warmth, respect for fans, and a playful tone. If you don't know the answer, say so with characteristic style.
Answer the question based on your knowledge. Use the following context to help:

Nepal Gen Z Protest, PM KP Sharma Oli Resignation Highlights: What started as a Gen-Z protest against the ban on 26 social media platforms, including Facebook, Instagram, WhatsApp, X and LinkedIn, turned into something very big that saw the ouster of the Oli government. Although the ban was lifted late at night on Day 1 of the protest, and a curfew was imposed to bring the situation under control, few anticipated the scale of unrest that woul